# A simple network to classify handwritten digits



Trong lab 3, chúng ta sẽ xây dựng những neural network đơn giản để phân loại các chữ số viết tay trong kho dữ liệu MNIST.

Phần 1: neural network này sẽ chỉ có input layer và output layer. Ta sẽ sử dụng softmax regression làm activation function cho output layer, và hàm cross entropy khi tính loss function. Với cách chọn hàm như vậy, network này hoàn toàn tương đồng với thuật toán logistic regression cho nhiều loại trong lab trước.

Phần 2: thêm vào neural network ở phần 1 một hidden layer ở giữa input và output layer.

Phần 3: bài tập thêm hidden layer nữa vào network ở phần 2.

Slide đi kèm: https://cloud.google.com/blog/big-data/2017/01/learn-tensorflow-and-deep-learning-without-a-phd.


## 1. Network với input và output layer
### 1.1 Khái quát
(Dựa theo https://www.tensorflow.org/get_started/mnist/beginners)
Trong phần này, chúng ta sẽ xây dựng một network không có hidden layer mà chỉ có input layer gồm 784 neuron được kết nối thẳng đến output layer gồm 10 neuron tương ứng với 10 nhóm chữ số viết tay.

TODO: Graph with 1 input and 1 output layer

Kết quả của 10 neuron này sẽ được biến đổi ở bước softmax regression để cuối cùng ta thu được 10 giá trị xác suất (là các số thực dương và tổng là 1) ứng với xác suất mỗi hình ảnh thuộc về từng nhóm chữ số. Cuối quá trình học, hình ảnh sẽ được network gắn cho nhóm chữ số mà xác suất hình ảnh này ứng với nhóm đó là cao nhất trong 10 giá trị xác suất.

Sai số giữa label dự đoán và label thật được tính bằng hàm cross entropy. Một lần nữa chúng ta sẽ dùng $GradientDescentOptimizer$ để thay đổi $weight$ và $bias$ giữa input layer và output layer, sao cho sai số này là nhỏ nhất.

### 1.2 Xây dựng network
Đầu tiên chúng ta hãy load dữ liệu bằng lệnh đã dùng trong lab trước. Lưu ý ta sẽ dùng $one\_hot$ vector cho labels. Hãy thử đoán xem dạng dữ liệu của labels thay đổi như thế nào với lựa chọn $one\_hot=True$?

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
mnist

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000000E3C3AEC50>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000000E3C3AEBE0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000000E3BDD7400>)

#### 1.2.1 Input

Hãy mô tả shape của các vector sau:
* input
* weight
* bias
* output

Và khởi tạo input, weight, bias. 

In [10]:
with tf.name_scope("Input") as scope:
    x = tf.placeholder(tf.float32, [None, 784], name="x-input")               # None means a dimension can be of any length
    y_correct = tf.placeholder(tf.float32, [None, 10], name="y-correct_label")
    
with tf.name_scope("Weight") as scope:
    W = tf.Variable(tf.zeros([784, 10]), name="weight")                       # weights
    
with tf.name_scope("Bias") as scope:
    b = tf.Variable(tf.zeros(10), name="bias")                                # bias


vectors (dimension, how to construct these vectors from our data set). What are the examples of incorrect prediction?

<b>answer:</b>

What could be the "loss function"? How many variables are there?

<b>answer:</b>

Why shouldn't we use the number of incorrectly classified images as the loss function?

<b>answer:</b>

[Mentors now show our way to construct a neural network for this problem: the use of cross entropy as the loss function, softmax regression as the output layer's activation function. Please note that this is just one of many different ways to solve this classification problem.]

Explain the shape of x, W, and b.

<b>answer:</b>

We will use softmax regression for the output layer.

In [11]:
with tf.name_scope("Softmax") as scope:
    y = tf.nn.softmax(tf.matmul(x, W) + b, name="softmax")

Cross entropy loss function

$H_{y'}(y) = -\sum_i y'_i \log(y_i)$

In [12]:
with tf.name_scope("Cross_Entropy") as scope:
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_correct * tf.log(y), reduction_indices=[1]), name="cross_entropy")

In [13]:
with tf.name_scope('Train') as scope:
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

Launch the model so far.

In [14]:
sess = tf.InteractiveSession()

In [15]:
file_writer = tf.summary.FileWriter("digit_classification_1_graphs", sess.graph)

Hãy nhìn vào graph thu được và hiểu trình tự của các operations.

![digit_classification_1_graph_1](https://github.com/chauvm/tensorflow_tutorials/raw/master/images/digit_classification_1_graph_1.png "TensorBoard's diagram of simple digit classification")

## Evaluating Our Model

Well, first let's figure out where we predicted the correct label. tf.argmax is an extremely useful function which gives you the index of the highest entry in a tensor along some axis. For example, $tf.argmax(y,\ 1)$ is the label our model thinks is most likely for each input, while $tf.argmax(y\_correct,\ 1)$ is the correct label. We can use tf.equal to check if our prediction matches the truth.

In [16]:
with tf.name_scope("Accuracy"):
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_correct,1), name = "correct_prediction")
#print(sess.run(correct_prediction, feed_dict={x: mnist.test.images,y_correct: mnist.test.labels}))

That gives us a list of booleans. To determine what fraction are correct, we cast to floating point numbers and then take the mean. For example, [True, False, True, True] would become [1,0,1,1] which would become 0.75.

In [17]:
with tf.name_scope("Accuracy"):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Chúng ta sẽ chạy bước train_step nhiều lần - tính sai số cross_entropy, dùng GradientDescentOptimizer để thay đổi trọng số và thiên lệch, và lặp lại. Với mỗi lần chạy, chúng ta nên lưu lại giá trị sai số và độ chính xác để thấy được chúng thay đổi thế nào trong quá trình học.

In [18]:
# create a summary for our cost and accuracy
tf.summary.scalar("cost_summary", cross_entropy)
tf.summary.scalar("accuracy", accuracy)

# merge all summaries into a single operation which we can execute in a session 
summary_step = tf.summary.merge_all()

Bây giờ ta có thể chạy thuật toán.

In [19]:
tf.global_variables_initializer().run()

In [20]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    _, summary = sess.run([train_step, summary_step], feed_dict={x: batch_xs,y_correct: batch_ys})
    
    # logging
    file_writer.add_summary(summary, i)

Hãy dùng TensorBoard để thấy đồ thị sự biến thiên của sai số và độ chính xác theo số lần học.


![digit_classification_1_graph_2](https://github.com/chauvm/tensorflow_tutorials/raw/master/images/digit_classification_1_graph_2.png "TensorBoard's diagram of simple digit classification")

![digit_classification_1_graph_3](https://github.com/chauvm/tensorflow_tutorials/raw/master/images/digit_classification_1_graph_3.png "TensorBoard's diagram of simple digit classification")

Độ chính xác của kết quả phân loại cuối cùng là:

In [24]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images,y_correct: mnist.test.labels}))

0.9177


Hãy thay đổi giá trị của số lần lặp và tốc độ học và quan sát kết quả thu được.

<b>answer:</b>

## 2. Network với hidden layers
